In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

import yaml
from utils import *

In [2]:
def spins_ani(lattice_spins, steps = STEPS, temp = T, size=N, J=J, KB=KB, B=B, display=True):

    total = np.sum(lattice_spins)
    m_values = np.zeros((steps))

    num_accept = 0
    for t in tqdm(range(steps)):
        i, j = np.random.randint(size), np.random.randint(size)
        delta_energy = 0
        for k, l in [(-1, 0), (1, 0), (0, 1), (0, -1)]:
            i_neigh = i + k if i + k < N else 0
            j_neigh = j + l if j + l < N else 0
            #delta_energy = np.sum(-J * -2 * lattice_spins[i, j] * mask[i, j]) - B * lattice_spins[i, j] <-- Dima implementation in old code pattern 
            delta_energy += -J * -2 * lattice_spins[i, j] * lattice_spins[i_neigh, j_neigh] #'-2' UNREMOVED
        delta_energy += - B * lattice_spins[i, j] #moved outside of the for loop; only acts once on the lattice point in question (i,j)
        if delta_energy <= 0:
            total -= lattice_spins[i, j]
            lattice_spins[i, j] *= -1
            total += lattice_spins[i, j]
            num_accept += 1
        else:
            prob = np.exp(-delta_energy / (KB * temp))
            if np.random.random() < prob:
                total -= lattice_spins[i, j]
                lattice_spins[i, j] *= -1
                total += lattice_spins[i, j]
                num_accept += 1
        m_values[t] = total
    return m_values/(size*size), lattice_spins, steps

In [4]:
plt.ioff()
fig, ax = plt.subplots(figsize=(10,10))

steps = 100
B = 0
lattice_spins = random_spins(100)

def update(frame):
    ax.clear()
    ax.set_xlabel('lattice')
    ax.set_title("2D ICING in time step = "+str(frame))  # Display frame number
    
    m_valuesUP, lattice_spins, steps = spins_ani(lattice_spins, steps=1)
    plot_lattice(lattice_spins)
    

ani = FuncAnimation(fig, update, frames=steps, interval=100)
#ani = FuncAnimation(fig, update, frames=steps, fargs=(img, config), blit=True, interval=50)
animation = ani
animation.save('animation.mp4', writer='ffmpeg', fps=30)
HTML(ani.to_jshtml())

MovieWriter ffmpeg unavailable; using Pillow instead.


IndexError: list index out of range

In [ ]:
ani = FuncAnimation(fig, update, frames=steps, fargs=(img, config), blit=True, interval=50)